In [1]:
import os
import sys
import argparse
import IPython 
from PIL import Image
from enum import Enum
from typing import Callable, List, Optional, Tuple, Union
from functools import partial

import torch
import torch.nn as nn
import torchvision
from torchvision.datasets import VisionDataset
from torchvision.transforms import transforms
import numpy as np
import pandas as pd
import skimage
from scipy import sparse
import matplotlib.pyplot as plt 
import torchxrayvision as xrv

from dinov2.models.unet import UNet
from dinov2.data import SamplerType, make_data_loader, make_dataset
from dinov2.data.datasets import NIHChestXray, MC
from dinov2.data.transforms import make_xray_classification_eval_transform, make_classification_eval_transform, make_segmentation_target_transform
from dinov2.eval.setup import setup_and_build_model
from dinov2.eval.utils import ModelWithIntermediateLayers, ModelWithNormalize, evaluate, extract_features
from dinov2.eval.metrics import build_multiclass_segmentation_metrics
from dinov2.utils import show_image_from_tensor

In [2]:
args = argparse.Namespace(config_file='dinov2/configs/eval/vits14_pretrain.yaml', pretrained_weights='models/dinov2_vits14_pretrain.pth', output_dir='results/NIH/dinov2_vits14/knn', opts=[], train_dataset_str='MC:split=TRAIN:root=/mnt/z/data/MC', val_dataset_str='MC:split=VAL:root=/mnt/z/data/MC', nb_knn=[5, 20, 50, 100, 200], temperature=0.07, gather_on_cpu=False, batch_size=8, n_per_class_list=[-1], n_tries=1, ngpus=1, nodes=1, timeout=2800, partition='learnlab', use_volta32=False, comment='', exclude='')
model, autocast_dtype = setup_and_build_model(args)
# model = ModelWithNormalize(model)

I20230824 02:15:55 8789 dinov2 config.py:60] git:
  sha: 3ecad4ab078af026e4d766fda5552a1e03850d4d, status: has uncommitted changes, branch: main

I20230824 02:15:55 8789 dinov2 config.py:61] batch_size: 8
comment: 
config_file: dinov2/configs/eval/vits14_pretrain.yaml
exclude: 
gather_on_cpu: False
n_per_class_list: [-1]
n_tries: 1
nb_knn: [5, 20, 50, 100, 200]
ngpus: 1
nodes: 1
opts: ['train.output_dir=/mnt/c/Users/user/Desktop/dinov2/results/NIH/dinov2_vits14/knn']
output_dir: /mnt/c/Users/user/Desktop/dinov2/results/NIH/dinov2_vits14/knn
partition: learnlab
pretrained_weights: models/dinov2_vits14_pretrain.pth
temperature: 0.07
timeout: 2800
train_dataset_str: MC:split=TRAIN:root=/mnt/z/data/MC
use_volta32: False
val_dataset_str: MC:split=VAL:root=/mnt/z/data/MC
I20230824 02:15:55 8789 dinov2 config.py:27] sqrt scaling learning rate; base: 0.004, new: 0.001
I20230824 02:15:55 8789 dinov2 config.py:34] MODEL:
  WEIGHTS: ''
compute_precision:
  grad_scaler: true
  teacher:
    backbon

In [3]:
train_dataset_str = args.train_dataset_str
val_dataset_str = args.val_dataset_str
batch_size = args.batch_size
gather_on_cpu = args.gather_on_cpu
num_workers = 0

In [4]:
transform = make_classification_eval_transform(resize_size=448, crop_size=448)
target_transform = make_segmentation_target_transform(resize_size=448)

train_dataset = make_dataset(
    dataset_str=args.train_dataset_str,
    transform=transform,
    target_transform=target_transform
)
val_dataset = make_dataset(
    dataset_str=args.val_dataset_str,
    transform=transform,
    target_transform=target_transform,
)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4)

I20230824 02:16:01 8789 dinov2 loaders.py:90] using dataset: "MC:split=TRAIN:root=/mnt/z/data/MC"
134 scans are missing from TRAIN set
I20230824 02:16:01 8789 dinov2 loaders.py:95] # of dataset samples: 138
I20230824 02:16:01 8789 dinov2 loaders.py:90] using dataset: "MC:split=VAL:root=/mnt/z/data/MC"
134 scans are missing from VAL set
I20230824 02:16:01 8789 dinov2 loaders.py:95] # of dataset samples: 138


In [5]:
class LinearDecoder(torch.nn.Module):
    def __init__(self, in_channels, tokenW=32, tokenH=32, num_labels=1):
        super(LinearDecoder, self).__init__()

        self.in_channels = in_channels
        self.width = tokenW
        self.height = tokenH
        self.decoder = torch.nn.Conv2d(in_channels, num_labels, (1,1))

    def forward(self, embeddings):
        embeddings = embeddings.reshape(-1, self.height, self.width, self.in_channels)
        embeddings = embeddings.permute(0,3,1,2)

        return self.decoder(embeddings)

In [6]:
decoder = LinearDecoder(384, num_labels=3).cuda()
optimizer = torch.optim.Adam(params=decoder.parameters(), lr=5e-4)

In [7]:
class MetricAveraging(Enum):
    MEAN_ACCURACY = "micro"
    MEAN_PER_CLASS_ACCURACY = "macro"
    MULTILABEL_ACCURACY = "macro"
    MULTILABEL_AUROC = "macro"
    MULTILABEL_JACCARD = "macro"
    PER_CLASS_ACCURACY = "none"

    def __str__(self):
        return self.value

metric = build_multiclass_segmentation_metrics(average_type=MetricAveraging.MULTILABEL_JACCARD,num_labels=3)
metric.cuda()

MetricCollection(
  (dice): Dice()
  (jaccard): MulticlassJaccardIndex()
)

In [19]:
model.embed_dim

384

In [11]:
for i in range(50):
    for image, target in train_loader:
        image, target = image.cuda(), target.cuda()
        with torch.no_grad(): 
            features=model.forward_features(image)['x_norm_patchtokens']
        logits = decoder(features)
        logits = torch.nn.functional.interpolate(logits, size=448, mode="bilinear", align_corners=False)
        prediction = logits.argmax(dim=1)

        loss_fct = torch.nn.CrossEntropyLoss(ignore_index=0)
        loss = loss_fct(logits, target)

        loss.backward()
        optimizer.step()

        optimizer.zero_grad()

        metric(prediction, target)
        print(metric.compute())
        print(loss.item())

KeyboardInterrupt: 